### 实际案例
    实现一个装饰器,它用来检查被装饰函数的参数类型，装饰器可以通过函数指明参数的类型,调用时如果检查出不匹配则抛出异常
    
    @typeassert(str, int, int)
    def f(a, b, c):
        ...
        
    @typeassert(y=list)
    def g(x,y):
        ...

### 解决方案
    带参数的装饰器,也就是根据参数定制化一个装饰器,可以看成生成装饰器的工厂,每次调用typeassert，返回一个特定的装饰器,然后用它去修饰其他函数

In [4]:
def typeassert(*ty_args, **ty_kargs):
    def decorator(func):
        # func -> a, b
        # d = {'a': int, 'b' : str}
        def wrapper(*args, **kargs):
            # args in d ,instance(arg, d[arg])
            return func(*args, **kargs)
        return wrapper
    return decorator

In [3]:
from inspect import signature

In [5]:
def f(a, b, c=1):pass

In [6]:
sig = signature(f)

In [7]:
sig.parameters

mappingproxy({'a': <Parameter "a">,
              'b': <Parameter "b">,
              'c': <Parameter "c=1">})

In [8]:
a = sig.parameters['a']

In [9]:
a.kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [10]:
a.default

inspect._empty

In [11]:
bargs = sig.bind(str, int, int)

In [12]:
bargs.arguments

OrderedDict([('a', str), ('b', int), ('c', int)])

In [13]:
bargs.arguments['a']

str

In [14]:
bargs.arguments['b']

int

In [15]:
sig.bind_partial(str)

<BoundArguments (a=<class 'str'>)>

### 案例解析代码

In [16]:
from inspect import signature
def typeassert(*ty_args, **ty_kargs):
    def decorator(func):
        # func -> a, b
        # d = {'a': int, 'b' : str}
        sig = signature(func)
        btypes = sig.bind_partial(*ty_args, **ty_kargs).arguments
        def wrapper(*args, **kargs):
            # args in d ,instance(arg, d[arg])
            for name, obj in sig.bind(*args, **kargs).arguments.items():
                if name in btypes:
                    if not isinstance(obj, btypes[name]):
                        raise TypeError('"%s" must be "%s"' % (name, btypes[name]))
            return func(*args, **kargs)
        return wrapper
    return decorator

In [17]:
@typeassert(int, str, list)
def f(a, b, c):
    print(a, b, c)

In [19]:
f(1,'abc',[1,2,3])

1 abc [1, 2, 3]


In [20]:
f(1,2,[1,2,3])

TypeError: "b" must be "<class 'str'>"